In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
from os.path import join
import os
import random
from functools import partial

import keras
from keras import layers
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential

from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.backend import binary_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard,RemoteMonitor
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

import ntpath
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#set_session(tf.Session(config=config))

K.set_image_dim_ordering('tf')

print(K.image_data_format())
smooth = 1e-12
num_examples = 0

import pandas as pd
import glob
import re
df = pd.read_csv('traindata.csv')


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


channels_last


In [2]:
df.head()

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False


In [3]:
boneage = df['boneage']
gender = df['male'].astype(int) # 0 means Female and 1 is Male

In [4]:
df.loc[df['id'] == 1380, 'male'].iloc[0].astype(int)

1

In [5]:
import glob
image_size = 512
orig_path = "/home/ubuntu/research/train/btrain/*"
orig = glob.glob(orig_path)
def load_files():
    global image_size
    global num_examples
    age_val=[]
    gender_val = []
    ori_images = []
    print("Starting Data Read")
    count = 0
    for fl in orig:
        count = count + 1
        orig_images= cv2.imread(fl, cv2.IMREAD_GRAYSCALE)
        orig_images = cv2.resize(orig_images, (image_size, image_size))
        a = int(re.findall('\d+', fl)[0])
        boneage = df.loc[df['id'] == a, 'boneage'].iloc[0]
        gender = df.loc[df['id'] == a, 'male'].iloc[0].astype(int)
        ori_images.append(orig_images)
        gender_val.append(gender)
        age_val.append(boneage)
        if count%50 == 0:
            break
            print("Data Read Percentage: "+str(round(((count*100)/len(orig)))))
    print("Data Loading Complete!")
    origi = np.array(ori_images)
    age = np.array(age_val)
    gender = np.array(gender_val)
    return origi,gender,age

origi, genders , ages = load_files()

Starting Data Read
Data Loading Complete!


In [ ]:
t_ori, v_ori , t_gender, v_gender, t_age, v_age = train_test_split(origi, genders, ages, test_size=0.15, random_state=42)

In [6]:
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
    """Utility function to apply conv + BN.

    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.

    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
   
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x


def Gender_InceptionV3(input_shape=(512, 512, 1)):
    """Instantiates the Inception v3 architecture.

    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format='channels_last'` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    Note that the default input image size for this model is 299x299.

    # Arguments
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(299, 299, 3)` (with `channels_last` data format)
            or `(3, 299, 299)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 139.
            E.g. `(150, 150, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.

    # Returns
        A Keras model instance.

    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    

    channel_axis = 3
    
    with tf.device('/cpu:0'):
        img_input = Input(shape=input_shape)
        x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
        
    with tf.device('/gpu:0'):
        x = conv2d_bn(x, 32, 3, 3, padding='valid')
        x = conv2d_bn(x, 64, 3, 3)
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        x = conv2d_bn(x, 80, 1, 1, padding='valid')
        x = conv2d_bn(x, 192, 3, 3, padding='valid')
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        # mixed 0, 1, 2: 35 x 35 x 256
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch5x5, branch3x3dbl, branch_pool],
            axis=channel_axis,
            name='mixed0')
        print(x)

    with tf.device('/gpu:1'):
        # mixed 1: 35 x 35 x 256
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch5x5, branch3x3dbl, branch_pool],
            axis=channel_axis,
            name='mixed1')

        # mixed 2: 35 x 35 x 256
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch5x5, branch3x3dbl, branch_pool],
            axis=channel_axis,
            name='mixed2')
        print(x)
        
    with tf.device('/gpu:2'):
        # mixed 3: 17 x 17 x 768
        branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

        branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = layers.concatenate(
            [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

        # mixed 4: 17 x 17 x 768
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 128, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 128, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch7x7, branch7x7dbl, branch_pool],
            axis=channel_axis,
            name='mixed4')
        print(x)

        # mixed 5, 6: 17 x 17 x 768
        for i in range(2):
            branch1x1 = conv2d_bn(x, 192, 1, 1)

            branch7x7 = conv2d_bn(x, 160, 1, 1)
            branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
            branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

            branch7x7dbl = conv2d_bn(x, 160, 1, 1)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

            branch_pool = AveragePooling2D(
                (3, 3), strides=(1, 1), padding='same')(x)
            branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
            x = layers.concatenate(
                [branch1x1, branch7x7, branch7x7dbl, branch_pool],
                axis=channel_axis,
                name='mixed' + str(5 + i))
        print(x)
            
            
    with tf.device('/gpu:3'):

        # mixed 7: 17 x 17 x 768
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 192, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 192, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate(
            [branch1x1, branch7x7, branch7x7dbl, branch_pool],
            axis=channel_axis,
            name='mixed7')
        print(x)
        
    with tf.device('/gpu:4'):

        # mixed 8: 8 x 8 x 1280
        branch3x3 = conv2d_bn(x, 192, 1, 1)
        branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                              strides=(2, 2), padding='valid')

        branch7x7x3 = conv2d_bn(x, 192, 1, 1)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

        branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = layers.concatenate(
            [branch3x3, branch7x7x3, branch_pool], axis=channel_axis, name='mixed8')
        print(x)
        
    with tf.device('/gpu:5'):
        # mixed 9: 8 x 8 x 2048
        for i in range(2):
            branch1x1 = conv2d_bn(x, 320, 1, 1)

            branch3x3 = conv2d_bn(x, 384, 1, 1)
            branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
            branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
            branch3x3 = layers.concatenate(
                [branch3x3_1, branch3x3_2], axis=channel_axis, name='mixed9_' + str(i))

            branch3x3dbl = conv2d_bn(x, 448, 1, 1)
            branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
            branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
            branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
            branch3x3dbl = layers.concatenate(
                [branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

            branch_pool = AveragePooling2D(
                (3, 3), strides=(1, 1), padding='same')(x)
            branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
            x = layers.concatenate(
                [branch1x1, branch3x3, branch3x3dbl, branch_pool],
                axis=channel_axis,
                name='mixed' + str(9 + i))
        print(x)
    
    with tf.device('/gpu:6'):
        x = Flatten()(x)
        y = layers.Input(shape=(1,))
        gender_dense = Dense(32, activation='relu',name='gender_dense')(y)
    
    with tf.device('/cpu:0'):
        x_merged = layers.concatenate([x,gender_dense])
        print(x_merged)
            # text_branch = Sequential()
            # text_branch.add(Dense(32, input_shape=(1,), activation='relu'))
            # merged = keras.layers.concatenate([x, text_branch.output])
            # This is taking the gender as input
            # 
        age = Dense(1000, activation='relu')(x_merged)
        age = Dense(1000, activation='relu')(age)


        age = Dense(1)(age)
        print(age)
        # gender_dense = Dense(32, activation='relu',name='gender_dense')(y)
        #x = layers.concatenate([gender_dense,x],axis=1)
        #x = Dense(1000, activation='relu', name='dense_1')(x)
        #x = Dense(1000, activation='relu', name='dense_2')(x)    
    # Create model.
    model = Model(inputs=[img_input, y], outputs=age, name='gender_inception_v3')


    return model

In [ ]:
class BatchGenerator:
    
    def __init__(self, size, batch_size):
        self.size = size
        self.batch_size = batch_size
        self.i = 0
    
    def get_batch(self):
        global t_ori
        global t_gender
        global t_age
        while True:
            orig_btch = np.zeros((self.batch_size, self.size, self.size, 1))
            gender_btch = np.zeros((self.batch_size, 1))
            age_btch = np.zeros((self.batch_size, 1))
            perm = np.arange(num_examples)
            np.random.shuffle(perm)
            some_age = [t_age[j] for j in perm]
            some_gender = [t_gender[j] for j in perm]
            some_ori = [t_ori[j] for j in perm]
            for i in range(self.batch_size):
                orig_btch[i] = some_ori[i]
                gender_btch[i] = some_gender[i]
                age_btch[i] = some_age[i]
            yield [orig_btch, gender_btch], age_btch

In [ ]:
class VBatchGenerator:
    
    def __init__(self, size, batch_size):
        self.size = size
        self.batch_size = batch_size
        self.i = 0
    
    def get_batch(self):
        global v_ori
        global v_gender
        global v_age
        while True:
            origbtch = np.zeros((self.batch_size, self.size, self.size, 1))
            genderbtch = np.zeros((self.batch_size, 1))
            agebtch = np.zeros((self.batch_size, 1))
            perm = np.arange(num_val)
            np.random.shuffle(perm)
            somev_age = [v_age[j] for j in perm]
            somev_gender = [v_gender[j] for j in perm]
            somev_ori = [v_ori[j] for j in perm]
            for i in range(self.batch_size):
                origbtch[i] = somev_ori[i]
                genderbtch[i] = somev_gender[i]
                agebtch[i] = somev_age[i]
            yield [origbtch, genderbtch], agebtch

In [15]:
tgi = np.resize(origi[1:2],(1,512,512,1)).astype(np.float32)
tga = ages[1:2].astype(np.float32)
tgg = genders[1:2].astype(np.float32)

In [ ]:
tgi = np.random.rand(4,512,512,3)
tga = np.random.rand(4,1)

In [13]:
tgi.shape

(512, 512, 1)

In [ ]:
tga = tga.astype(np.float32)

In [ ]:
tgg

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
import keras
from keras.models import Model, Sequential
from keras.models import Input
input_shape = (512, 512, 3)
base_model = VGG16(input_shape=input_shape, weights=None, include_top=False)

pixel_branch = base_model.output
pixel_branch = Flatten()(pixel_branch)

new_model = Model(inputs=base_model.input, outputs=pixel_branch)

text_branch = Sequential()
text_branch.add(Dense(32, input_shape=(1,), activation='relu'))

# merged = Merge([new_model, text_branch], mode='concat')
merged = keras.layers.concatenate([new_model.output, text_branch.output])

age = Dense(1000, activation='relu')(merged)
age = Dense(1000, activation='relu')(age)
age = Dense(1)(age)

# show model
# model.summary()
model = Model(inputs=[base_model.input, text_branch.input], outputs=age)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import keras
from keras.models import Model
from keras.models import Sequential

from keras.models import Input
input_shape = (512, 512, 3)
base_model = InceptionV3(input_shape=input_shape, weights=None, include_top=False)

pixel_branch = base_model.output
pixel_branch = Flatten()(pixel_branch)

new_model = Model(inputs=base_model.input, outputs=pixel_branch)

text_branch = Sequential()
text_branch.add(Dense(32, input_shape=(1,), activation='relu'))

# merged = Merge([new_model, text_branch], mode='concat')
merged = keras.layers.concatenate([new_model.output, text_branch.output])

age = Dense(1000, activation='relu')(merged)
age = Dense(1000, activation='relu')(age)
age = Dense(1)(age)

# show model
# model.summary()
model = Model(inputs=[base_model.input, text_branch.input], outputs=age)

In [8]:
model = Gender_InceptionV3()
model.summary()

Tensor("mixed0/concat:0", shape=(?, 61, 61, 256), dtype=float32, device=/device:GPU:0)
Tensor("mixed2/concat:0", shape=(?, 61, 61, 288), dtype=float32, device=/device:GPU:1)
Tensor("mixed4/concat:0", shape=(?, 30, 30, 768), dtype=float32, device=/device:GPU:2)
Tensor("mixed6/concat:0", shape=(?, 30, 30, 768), dtype=float32, device=/device:GPU:2)
Tensor("mixed7/concat:0", shape=(?, 30, 30, 768), dtype=float32, device=/device:GPU:3)
Tensor("mixed8/concat:0", shape=(?, 14, 14, 1280), dtype=float32, device=/device:GPU:4)
Tensor("mixed10/concat:0", shape=(?, 14, 14, 2048), dtype=float32, device=/device:GPU:5)
Tensor("concatenate_3/concat:0", shape=(?, ?), dtype=float32, device=/device:CPU:0)
Tensor("dense_3/BiasAdd:0", shape=(?, 1), dtype=float32, device=/device:CPU:0)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)       

In [9]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes
print(get_model_memory_usage(1,model))

1.95


In [ ]:
from time import time
from keras import optimizers
 

# compile mode
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) 
model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['accuracy'])
model.fit([tgi,tgg],tga,1,epochs = 500, verbose=1)

Epoch 1/500


In [13]:
model.predict([tgi[0].reshape(1,256,256,3),tgg[0].reshape(1,1)])

array([[97.668976]], dtype=float32)

In [ ]:
from time import time
from keras import optimizers

num_examples = t_ori.shape[0]
num_val = v_ori.shape[0]
batch_size = 16
s_tr =np.floor(float(t_ori.shape[0]) / float(batch_size))
s_val =np.floor(float(v_ori.shape[0]) / float(batch_size))
train_batch_generator = BatchGenerator( image_size, batch_size)
valid_batch_generator = VBatchGenerator(image_size, batch_size)
size = image_size
n_cls = 1


optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) 

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001) 
model = Gender_InceptionV3()
model.compile(optimizer=optim, loss='mean_absolute_error', metrics=['acc'])
tensorboard = TensorBoard(log_dir="reglog/", batch_size=16,write_graph=True)
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             RemoteMonitor(root='http://localhost:9000', path='/publish/epoch/end/', field='data', headers=None),
             tensorboard]
model.fit_generator(train_batch_generator.get_batch(),steps_per_epoch=s_tr,epochs=500, callbacks=callbacks, validation_data=valid_batch_generator.get_batch(),validation_steps=s_val)

In [ ]:
q = train_batch_generator.get_batch()

In [ ]:
a,b=q.__next__()

In [ ]:
w = a[0]

In [ ]:
jh=w[0].reshape(500,500)

In [ ]:
b[0]

In [ ]:
from importlib import reload
reload(K)
import h5py
model.save('regression.h5')

In [ ]:
image = v_ori[0].reshape(1,500,500,1)

In [ ]:
image.shape

In [ ]:
gender= v_gender[0].reshape(1,1)

In [ ]:
q = model.predict([image,gender])

In [ ]:
q

In [ ]:
plt.imshow(t_ori[1])

In [ ]:
t_gender[1]

In [ ]:
t_age[1]

In [ ]:
t_age[2]